In [ ]:
#@title 🔧 Setup: clonar repo e instalar librerías
!git clone --depth 1 https://github.com/IDEA-Research/Grounded-SAM-2.git
%cd Grounded-SAM-2

# libs básicas + SAM-2 oficial
!pip -q install supervision opencv-python pillow pycocotools transformers accelerate einops ninja
!pip -q install git+https://github.com/facebookresearch/sam2.git

Cloning into 'Grounded-SAM-2'...
remote: Enumerating objects: 925, done.
remote: Counting objects: 100% (925/925), done.
remote: Compressing objects: 100% (783/783), done.
remote: Total 925 (delta 122), reused 854 (delta 118), pack-reused 0 (from 0)
Receiving objects: 100% (925/925), 112.02 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Updating files: 100% (839/839), done.
/content/Grounded-SAM-2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [ ]:
#@title ⬇️ Descarga de modelo e imagen
import os, urllib.request, pathlib, shutil, zipfile, json, torch, cv2, numpy as np
from itertools import islice
from tqdm import tqdm
from pathlib import Path

# Carpeta de checkpoints local
ckpt_dir = Path("checkpoints"); ckpt_dir.mkdir(exist_ok=True)
sam2_ckpt_path = ckpt_dir / "sam2_hiera_large.pt"

# ≈900 MB – tarda ~1-2 min si no está en caché
if not sam2_ckpt_path.exists():
    !wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -O checkpoints/sam2_hiera_large.pt

# Imagen aérea gratuita (Unsplash)
#img_url = "https://storage.googleapis.com/support-forums-api/attachment/thread-264444595-7228703151249223747.png"
#img_path = "satellite.jpg"
#urllib.request.urlretrieve(img_url, img_path)

#Cargar imagen en local
from google.colab import files
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

#Cargar carpeta de drive
#from google.colab import drive
#drive.mount('/content/drive')

#Cargar carpeta de imagenes
#input_folder = "/content/Imagenes_segmentar"
#output_folder = "segmentados_output"
#os.makedirs(output_folder, exist_ok=True)

print("✔ Recursos listos")

Saving Florida.png to Florida.png
✔ Recursos listos


In [ ]:
sam2_ckpt_path

PosixPath('checkpoints/sam2_hiera_large.pt')

In [ ]:
ckpt_dir = Path("checkpoints"); ckpt_dir.mkdir(exist_ok=True)
sam2_ckpt_path = "sam2.1_hiera_large.pt"
ckpt_path = ckpt_dir / sam2_ckpt_path
if not ckpt_path.exists():
    !wget -q \
      https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt \
      -O {ckpt_path}

In [ ]:
input_folder

'/content/Imagenes_segmentar'

In [ ]:
img_path

'tile_11505_14160.png'

In [ ]:
#@title 🚀 Inferencia Grounded-SAM-2 para retornar una carpeta de imagenes
import supervision as sv
from supervision.draw.color import ColorPalette
import pycocotools.mask as mask_util
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch

# Parámetros
TEXT_PROMPT = "trees shrubs. trees. natural vegetation. parks" #"natural vegetation. grass. trees. shrubs. parks. gardens. lawns"
SAM2_CFG    = "configs/sam2.1/sam2.1_hiera_l.yaml"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

# SAM-2
sam2_model = build_sam2(SAM2_CFG, "checkpoints/"+str(sam2_ckpt_path), device=DEVICE)
sam2_pred  = SAM2ImagePredictor(sam2_model)

# Grounding DINO vía HF
gd_model_id = "IDEA-Research/grounding-dino-base"
processor   = AutoProcessor.from_pretrained(gd_model_id)
gd_model    = AutoModelForZeroShotObjectDetection.from_pretrained(gd_model_id).to(DEVICE)

import json  # <-- añadido para guardar los datos
from tqdm import tqdm  # para mostrar barra de progreso

# 🔁Itera sobre cada imagen
# Procesar en paquetes de 50
for filename in tqdm(os.listdir(input_folder)):
    img_path = os.path.join(input_folder, filename)
    image = Image.open(img_path).convert("RGB")
    sam2_pred.set_image(np.array(image))

    # Grounding DINO → cajas
    inputs = processor(images=image, text=TEXT_PROMPT, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        out = gd_model(**inputs)

    result = processor.post_process_grounded_object_detection(
        out, inputs.input_ids,
        box_threshold=0.2, text_threshold=0.19,
        target_sizes=[image.size[::-1]]
    )[0]

    boxes   = result["boxes"].cpu().numpy()
    labels  = result["labels"]
    scores  = result["scores"].cpu().numpy().tolist()
    class_ids = np.arange(len(labels))

    # SAM-2 → máscaras
    masks, _, _ = sam2_pred.predict(
        box=boxes, point_coords=None, point_labels=None,
        multimask_output=False,
    )
    if masks.ndim == 4:
        masks = masks.squeeze(1)

    # 📦 Filtrado por tamaño de caja
    img_w, img_h = image.size
    image_area = img_w * img_h
    max_area_ratio = 0.3

    filtered_boxes = []
    filtered_labels = []
    filtered_scores = []
    valid_indices = []

    for i, (box, label, score) in enumerate(zip(boxes, labels, scores)):
        x1, y1, x2, y2 = box
        box_area = (x2 - x1) * (y2 - y1)
        if box_area / image_area <= max_area_ratio:
            filtered_boxes.append(box.tolist())  # convierte a lista para JSON
            filtered_labels.append(label)
            filtered_scores.append(score)
            valid_indices.append(i)

    filtered_boxes = np.array(filtered_boxes)
    masks = masks[valid_indices]
    class_ids = np.arange(len(filtered_labels))

    # 🎨 Visualización
    frame = cv2.imread(img_path)
    dets = sv.Detections(xyxy=filtered_boxes, mask=masks.astype(bool), class_id=class_ids)
    from supervision.draw.color import ColorPalette, Color
    try:
        palette = ColorPalette.default()
    except AttributeError:
        palette = ColorPalette([
            Color(20, 200, 50)
        ])

    frame = sv.BoxAnnotator(color=palette).annotate(frame, dets)
    frame = sv.LabelAnnotator(color=palette).annotate(frame, dets,
             labels=[f"{n} {s:.2f}" for n, s in zip(filtered_labels, filtered_scores)])
    frame = sv.MaskAnnotator(color=palette).annotate(frame, dets)

    # Guarda imagen segmentada
    #output_img_path = os.path.join(output_folder, filename)
    #cv2.imwrite(output_img_path, frame)

    json_data = {"masks": []}

    for i in range(len(masks)):
        mask = masks[i].astype(np.uint8)
        rle = mask_util.encode(np.asfortranarray(mask))
        rle["counts"] = rle["counts"].decode("utf-8")  # JSON friendly

        json_data["masks"].append({
            "segmentation": rle,
            "label": str(filtered_labels[i]),
            "score": float(filtered_scores[i])
        })

    # Guardar el JSON con el mismo nombre de la imagen
    json_filename = os.path.splitext(filename)[0] + ".json"
    json_path = os.path.join(output_folder, json_filename)

    with open(json_path, "w") as f:
        json.dump(json_data, f, indent=2)

    print(f"✅ {filename} procesado, imagen y JSON guardados.")

  0%|          | 0/4 [00:01<?, ?it/s]


ValueError: xyxy must be a 2D np.ndarray with shape (_, 4), but got shape (0,)

In [ ]:
#@title Descargar carpeta de segmentados_output
import shutil
shutil.make_archive("output_mask_folder", 'zip', "/content/output_mask_folder")

from google.colab import files
files.download("output_mask_folder.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Convertir json en mascara binaria
import json
import numpy as np
from pycocotools import mask as maskUtils
from PIL import Image

# Cargar archivo JSON
for filename in os.listdir("/content/json_folder"):
  if filename.endswith(".json"):
    json_path = os.path.join("/content/json_folder", filename)
    with open(json_path, "r") as f:
        data = json.load(f)

    # Obtener tamaño
    height, width = data["masks"][0]["segmentation"]["size"]
    mask_total = np.zeros((height, width), dtype=np.uint8)

    # Decodificar cada máscara y combinarlas
    for mask in data["masks"]:
        rle = mask["segmentation"]
        binary_mask = maskUtils.decode(rle)
        mask_total = np.maximum(mask_total, binary_mask.astype(np.uint8))

    # Guardar máscara binaria
    base_name = os.path.splitext(filename)[0]
    output_path = os.path.join("/content/output_mask_folder", f"{base_name}_mascara_binaria.png")
    Image.fromarray(mask_total * 255).save(output_path)

    print(f"✅ Máscara binaria guardada como: {output_path}")

✅ Máscara binaria guardada como: /content/output_mask_folder/tile_13275_16815_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_11505_11505_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_8850_26550_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_23010_20355_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_885_20355_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_4425_13275_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_18585_18585_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_12390_6195_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_18585_12390_mascara_binaria.png
✅ Máscara binaria guardada como: /content/output_mask_folder/tile_1770_18585_mascara_binaria.png
✅ Máscara binaria guardada

In [ ]:
#@title 🚀 Inferencia Grounded-SAM-2 para retornar json de una carpeta del drive
import supervision as sv
from supervision.draw.color import ColorPalette
import pycocotools.mask as mask_util
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch

# Parámetros
TEXT_PROMPT = "trees shrubs. trees. natural vegetation. parks. vegetation. forest" #"natural vegetation. grass. trees. shrubs. parks. gardens. lawns"
SAM2_CFG    = "configs/sam2.1/sam2.1_hiera_l.yaml"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

# Rutas en tu Google Drive
input_folder = "/content/drive/MyDrive/images"
output_folder = "/content/drive/MyDrive/imagenes_json"
os.makedirs(output_folder, exist_ok=True)

# SAM-2 y GroundingDINO
sam2_model = build_sam2(SAM2_CFG, "checkpoints/"+str(sam2_ckpt_path), device=DEVICE)
sam2_pred = SAM2ImagePredictor(sam2_model)

gd_model_id = "IDEA-Research/grounding-dino-base"
processor = AutoProcessor.from_pretrained(gd_model_id)
gd_model = AutoModelForZeroShotObjectDetection.from_pretrained(gd_model_id).to(DEVICE)

# Procesar cada imagen
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png'))]

for filename in tqdm(image_files, desc="Procesando imágenes"):
    img_path = os.path.join(input_folder, filename)
    image = Image.open(img_path).convert("RGB")
    sam2_pred.set_image(np.array(image))

    # DINO
    inputs = processor(images=image, text=TEXT_PROMPT, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        out = gd_model(**inputs)

    result = processor.post_process_grounded_object_detection(
        out, inputs.input_ids,
        box_threshold=0.2, text_threshold=0.19,
        target_sizes=[image.size[::-1]]
    )[0]

    boxes = result["boxes"].cpu().numpy()
    labels = result["labels"]
    scores = result["scores"].cpu().numpy().tolist()
    class_ids = np.arange(len(labels))

    img_w, img_h = image.size
    image_area = img_w * img_h
    max_area_ratio = 0.7

    filtered_boxes = []
    filtered_labels = []
    filtered_scores = []
    valid_indices = []

    for i, (box, label, score) in enumerate(zip(boxes, labels, scores)):
        x1, y1, x2, y2 = box
        box_area = (x2 - x1) * (y2 - y1)
        if box_area / image_area <= max_area_ratio:
            filtered_boxes.append(box)
            filtered_labels.append(label)
            filtered_scores.append(score)
            valid_indices.append(i)

    boxes = np.array(filtered_boxes)
    labels = filtered_labels
    scores = filtered_scores
    class_ids = np.arange(len(labels))

    if len(filtered_boxes) == 0:
        print(f"🔔 Sin cajas válidas para {filename}. Saltando...")
        continue
    # SAM-2
    masks, _, _ = sam2_pred.predict(
        box=boxes, point_coords=None, point_labels=None,
        multimask_output=False,
    )
    if masks.ndim == 4:
        masks = masks.squeeze(1)
    #masks = masks[valid_indices]

    # Guardar JSON
    json_data = {"masks": []}
    for i in range(len(masks)):
        mask = masks[i].astype(np.uint8)
        rle = mask_util.encode(np.asfortranarray(mask))
        rle["counts"] = rle["counts"].decode("utf-8")  # para JSON

        json_data["masks"].append({
            "segmentation": rle,
            "label": str(labels[i]),
            "score": float(scores[i])
        })

    json_filename = os.path.splitext(filename)[0] + ".json"
    json_path = os.path.join(output_folder, json_filename)
    with open(json_path, "w") as f:
        json.dump(json_data, f, indent=2)

    print(f"✅ {filename} procesado y guardado como {json_filename}")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

✅ tile_885_21240.png procesado y guardado como tile_885_21240.json


Procesando imágenes: 100%|█████████▉| 447/448 [10:54<00:01,  1.51s/it]

✅ tile_4425_13275.png procesado y guardado como tile_4425_13275.json


Procesando imágenes: 100%|██████████| 448/448 [10:55<00:00,  1.46s/it]

✅ tile_2655_19470.png procesado y guardado como tile_2655_19470.json


In [ ]:
#@title 🚀 Inferencia Grounded-SAM-2 para retornar json de una carpeta del drive (Para imagenes restantes)
import supervision as sv
from supervision.draw.color import ColorPalette
import pycocotools.mask as mask_util
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch

# Parámetros
TEXT_PROMPT = "trees shrubs. trees. natural vegetation. parks. vegetation. forest. trees" #"natural vegetation. grass. trees. shrubs. parks. gardens. lawns"
SAM2_CFG    = "configs/sam2.1/sam2.1_hiera_l.yaml"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

# Rutas en tu Google Drive
input_folder = "/content/drive/MyDrive/images"
output_folder = "/content/drive/MyDrive/imagenes_json"
os.makedirs(output_folder, exist_ok=True)

# Obtener nombres base sin extensión
imagenes_input = {os.path.splitext(f)[0] for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))}
json_generados = {os.path.splitext(f)[0] for f in os.listdir(output_folder) if f.endswith('.json')}

# Comparar
imagenes_faltantes = imagenes_input - json_generados

print("📌 Imágenes pendientes de procesar:")
for nombre in sorted(imagenes_faltantes):
    print(f"- {nombre}")

# SAM-2 y GroundingDINO
sam2_model = build_sam2(SAM2_CFG, "checkpoints/"+str(sam2_ckpt_path), device=DEVICE)
sam2_pred = SAM2ImagePredictor(sam2_model)

gd_model_id = "IDEA-Research/grounding-dino-base"
processor = AutoProcessor.from_pretrained(gd_model_id)
gd_model = AutoModelForZeroShotObjectDetection.from_pretrained(gd_model_id).to(DEVICE)

# Procesar cada imagen
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png'))]

for nombre_base in imagenes_faltantes:
    img_path = os.path.join(input_folder, nombre_base + ".png")
    image = Image.open(img_path).convert("RGB")
    sam2_pred.set_image(np.array(image))

    # DINO
    inputs = processor(images=image, text=TEXT_PROMPT, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        out = gd_model(**inputs)

    result = processor.post_process_grounded_object_detection(
        out, inputs.input_ids,
        box_threshold=0.2, text_threshold=0.19,
        target_sizes=[image.size[::-1]]
    )[0]

    boxes = result["boxes"].cpu().numpy()
    labels = result["labels"]
    scores = result["scores"].cpu().numpy().tolist()
    class_ids = np.arange(len(labels))

    img_w, img_h = image.size
    image_area = img_w * img_h
    max_area_ratio = 0.9

    filtered_boxes = []
    filtered_labels = []
    filtered_scores = []
    valid_indices = []

    for i, (box, label, score) in enumerate(zip(boxes, labels, scores)):
        x1, y1, x2, y2 = box
        box_area = (x2 - x1) * (y2 - y1)
        if box_area / image_area <= max_area_ratio:
            filtered_boxes.append(box)
            filtered_labels.append(label)
            filtered_scores.append(score)
            valid_indices.append(i)

    boxes = np.array(filtered_boxes)
    labels = filtered_labels
    scores = filtered_scores
    class_ids = np.arange(len(labels))
    if len(filtered_boxes) == 0:
        print(f"🔔 Sin cajas válidas para {nombre_base}. Saltando...")
        continue
    # SAM-2
    masks, _, _ = sam2_pred.predict(
        box=boxes, point_coords=None, point_labels=None,
        multimask_output=False,
    )
    if masks.ndim == 4:
        masks = masks.squeeze(1)
    #masks = masks[valid_indices]

    # Guardar JSON
    json_data = {"masks": []}
    for i in range(len(masks)):
        mask = masks[i].astype(np.uint8)
        rle = mask_util.encode(np.asfortranarray(mask))
        rle["counts"] = rle["counts"].decode("utf-8")  # para JSON

        json_data["masks"].append({
            "segmentation": rle,
            "label": str(labels[i]),
            "score": float(scores[i])
        })

    json_filename = os.path.splitext(filename)[0] + ".json"
    json_path = os.path.join(output_folder, json_filename)
    with open(json_path, "w") as f:
        json.dump(json_data, f, indent=2)

    print(f"✅ {nombre_base} procesado y guardado como {json_filename}")

📌 Imágenes pendientes de procesar:
- tile_10620_4425
- tile_13275_13275
- tile_3540_18585
- tile_7080_6195
🔔 Sin cajas válidas para tile_10620_4425. Saltando...
🔔 Sin cajas válidas para tile_13275_13275. Saltando...
✅ tile_3540_18585 procesado y guardado como tile_2655_19470.json
🔔 Sin cajas válidas para tile_7080_6195. Saltando...


In [ ]:
#@title 🚀 Inferencia Grounded-SAM-2 para retornar una imagen RGB segmentada
import supervision as sv
from supervision.draw.color import ColorPalette
import pycocotools.mask as mask_util
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch

# Parámetros
TEXT_PROMPT = "trees shrubs. trees. natural vegetation. vegetation. forest" #"natural vegetation. grass. trees. shrubs. parks. gardens. lawns"
SAM2_CFG    = "configs/sam2.1/sam2.1_hiera_l.yaml"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

# SAM-2
sam2_model = build_sam2(SAM2_CFG, "checkpoints/"+str(sam2_ckpt_path), device=DEVICE)
sam2_pred  = SAM2ImagePredictor(sam2_model)

# Grounding DINO vía HF
gd_model_id = "IDEA-Research/grounding-dino-base"
processor   = AutoProcessor.from_pretrained(gd_model_id)
gd_model    = AutoModelForZeroShotObjectDetection.from_pretrained(gd_model_id).to(DEVICE)

# Imagen
image = Image.open(img_path).convert("RGB")
sam2_pred.set_image(np.array(image))

#Ubicar json en colab
output_folder = "/content/"
os.makedirs(output_folder, exist_ok=True)
filename = os.path.basename(img_path)

# DINO → cajas
inputs  = processor(images=image, text=TEXT_PROMPT, return_tensors="pt").to(DEVICE)
with torch.no_grad():
    out = gd_model(**inputs)

result = processor.post_process_grounded_object_detection(
    out, inputs.input_ids,
    box_threshold=0.14, text_threshold=0.2,
    target_sizes=[image.size[::-1]]
)[0]

print(result)

boxes   = result["boxes"].cpu().numpy()           # (n,4)
labels  = result["labels"]
scores  = result["scores"].cpu().numpy().tolist()
class_ids = np.arange(len(labels))

#print("boxes",boxes)
print("labels",set(labels))

img_w, img_h = image.size
image_area = img_w * img_h
max_area_ratio = 0.3

filtered_boxes = []
filtered_labels = []
filtered_scores = []

for box, label, score in zip(boxes, labels, scores):
    x1, y1, x2, y2 = box
    box_area = (x2 - x1) * (y2 - y1)

    if box_area / image_area <= max_area_ratio:
        filtered_boxes.append(box)
        filtered_labels.append(label)
        filtered_scores.append(score)

# Convierte a numpy arrays
boxes = np.array(filtered_boxes)
labels = filtered_labels
scores = filtered_scores
class_ids = np.arange(len(labels))

# SAM-2 → máscaras
masks, _, _ = sam2_pred.predict(
    box=boxes, point_coords=None, point_labels=None,
    multimask_output=False,
)
if masks.ndim == 4: masks = masks.squeeze(1)

# Visualización
frame = cv2.imread(img_path)
dets  = sv.Detections(xyxy=boxes, mask=masks.astype(bool), class_id=class_ids)

from supervision.draw.color import ColorPalette, Color

try:                                   # usa API nueva si existe
    palette = ColorPalette.default()
except AttributeError:                 # retro-compatibilidad
    palette = ColorPalette([           # paleta básica manual
        Color(20, 200, 50)
    ])
#frame  = sv.BoxAnnotator(color=palette).annotate(frame, dets)
#frame  = sv.LabelAnnotator(color=palette).annotate(frame, dets,
#         labels=[f"{n} {s:.2f}" for n,s in zip(labels,scores)])
frame  = sv.MaskAnnotator(color=palette).annotate(frame, dets)
cv2.imwrite("grounded_sam2_output.jpg", frame)
# Guardar JSON con RLE
json_data = {"masks": []}
for i in range(len(masks)):
    mask = masks[i].astype(np.uint8)
    rle = mask_util.encode(np.asfortranarray(mask))
    rle["counts"] = rle["counts"].decode("utf-8")
    json_data["masks"].append({
        "segmentation": rle,
        "label": str(labels[i]),
        "score": float(scores[i])
    })

json_filename = os.path.splitext(filename)[0] + ".json"
json_path = os.path.join(output_folder, json_filename)
with open(json_path, "w") as f:
    json.dump(json_data, f, indent=2)

print(f"✅ {filename} procesado y guardado como {json_filename}")
print(f"✅ Imagen anotada guardada en: {os.path.join(output_folder, 'grounded_sam2_output.jpg')}")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

{'scores': tensor([0.4208, 0.3928, 0.3130, 0.3695, 0.3514, 0.3632, 0.2985, 0.3206, 0.3134,
        0.3241, 0.2594, 0.3049, 0.1845, 0.2640, 0.2770, 0.2433, 0.2725, 0.2822,
        0.2744, 0.2665, 0.2517, 0.2612, 0.3186, 0.3460, 0.2167, 0.2726, 0.2229,
        0.1893, 0.2367, 0.1884, 0.2366, 0.2097, 0.2268, 0.1639, 0.2249, 0.1981,
        0.2177, 0.2485, 0.1549, 0.1530, 0.2038, 0.2685, 0.1537, 0.1846, 0.1555,
        0.2788, 0.1878, 0.1737, 0.1678, 0.1842, 0.1493, 0.1567, 0.2049, 0.1988,
        0.1624, 0.1536, 0.2317, 0.1736, 0.1460, 0.1486, 0.1770, 0.2013, 0.1716,
        0.1850, 0.1886, 0.1581, 0.1516, 0.1437, 0.1478, 0.1490, 0.1504, 0.1574,
        0.1459, 0.1839, 0.1975, 0.1486, 0.2035, 0.1651, 0.1765, 0.1599, 0.1618,
        0.1670, 0.1666, 0.1579, 0.1762, 0.1947, 0.1748, 0.1597, 0.1541, 0.2172,
        0.1586, 0.1926, 0.2038, 0.1790, 0.1722, 0.1478, 0.1521, 0.1838, 0.1495,
        0.1538, 0.1428], device='cuda:0'), 'boxes': tensor([[1.6772e+02, 1.9624e+02, 2.1666e+02, 2.4915e+02],